In [5]:
import pandas as pd

DATA_PATH = "/kaggle/input/diversevul-20230702/diversevul_20230702.json"

import json

rows = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

df = pd.DataFrame(rows)
print(df.shape)
print(df.columns)
df.head()



(330492, 8)
Index(['func', 'target', 'cwe', 'project', 'commit_id', 'hash', 'size',
       'message'],
      dtype='object')


,func,target,cwe,project,commit_id,hash,size,message
0,int _gnutls_ciphertext2compressed(gnutls_sessi...,1,[],gnutls,7ad6162573ba79a4392c63b453ad0220ca6c5ace,73008646937836648589283922871188272089,157,added an extra check while checking the padding.
1,static char *make_filename_safe(const char *fi...,1,[CWE-264],php-src,055ecbc62878e86287d742c7246c21606cee8183,211824207069112513181516095447837228041,22,Improve check for :memory: pseudo-filename in ...
2,"unpack_Z_stream(int fd_in, int fd_out)\n{\n\tI...",1,[],busybox,251fc70e9722f931eec23a34030d05ba5f747b0e,21401706257394042943815500829552774160,232,uncompress: fix buffer underrun by corrupted i...
3,"static void cirrus_do_copy(CirrusVGAState *s, ...",1,[CWE-787],qemu,b2eb849d4b1fdb6f35d5c46958c7f703cf64cfef,135590882627853658533498335902319684573,66,"CVE-2007-1320 - Cirrus LGD-54XX ""bitblt"" heap ..."
4,"glue(cirrus_bitblt_rop_fwd_, ROP_NAME)(CirrusV...",1,[CWE-787],qemu,b2eb849d4b1fdb6f35d5c46958c7f703cf64cfef,27696392987383562433164405181263025184,18,"CVE-2007-1320 - Cirrus LGD-54XX ""bitblt"" heap ..."


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# assumes df already exists from your load step
df = df.dropna(subset=["func","target"]).copy()
df["target"] = df["target"].astype(int)

X = df["func"].astype(str)
y = df["target"].astype(int)

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=42, stratify=y_tmp)

len(X_train), len(X_val), len(X_test), y_train.mean()


(231344, 49574, 49574, 0.05732156442354243)

In [7]:
# Install transformers if needed
try:
    import transformers  # noqa
except Exception:
    !pip -q install transformers

import torch, os, math
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base").to(device)
model.eval();  # freeze
for p in model.parameters(): p.requires_grad = False

MAX_TOKENS = 128     # keep small; raise to 256 later if you want
WINDOW = 3           # sliding window size
BATCH = 16           # adjust based on VRAM/RAM

os.makedirs("/kaggle/working/graphs/train", exist_ok=True)
os.makedirs("/kaggle/working/graphs/val", exist_ok=True)
os.makedirs("/kaggle/working/graphs/test", exist_ok=True)

def build_edges(n_tokens:int, win:int=WINDOW):
    # undirected edges within [i, i+1..i+win-1]
    src, dst = [], []
    for i in range(n_tokens):
        for j in range(i+1, min(n_tokens, i+win)):
            src += [i, j]; dst += [j, i]
    if not src:
        return torch.empty(2,0, dtype=torch.long)
    return torch.tensor([src, dst], dtype=torch.long)

def save_graph_batch(texts, labels, out_dir, start_idx):
    enc = tokenizer(texts, padding=True, truncation=True, max_length=MAX_TOKENS, return_tensors="pt")
    enc = {k:v.to(device) for k,v in enc.items()}
    with torch.no_grad():
        out = model(**enc).last_hidden_state  # [B, T, 768]
    attn = enc["attention_mask"].cpu()       # [B, T]

    for i in range(out.size(0)):
        tlen = int(attn[i].sum().item())
        x = out[i, :tlen].half().cpu()       # [tlen, 768] fp16 to save space
        edge_index = build_edges(tlen)       # [2, E]
        y = torch.tensor([int(labels[i])], dtype=torch.long)
        idx = start_idx + i
        torch.save({"x": x, "edge_index": edge_index, "y": y},
                   os.path.join(out_dir, f"g_{idx}.pt"))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

2025-08-10 07:25:23.170570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754810723.519428      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754810723.619383      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [8]:
from math import ceil
from tqdm import trange

# how many samples to process now
N_TRAIN, N_VAL, N_TEST = 10_000, 2_000, 2_000

def process_split(Xseries, yseries, out_dir, limit):
    texts = Xseries.iloc[:limit].tolist()
    labels = yseries.iloc[:limit].tolist()
    total = len(texts)
    for s in trange(0, total, BATCH):
        e = min(s+BATCH, total)
        save_graph_batch(texts[s:e], labels[s:e], out_dir, start_idx=s)

process_split(X_train, y_train, "/kaggle/working/graphs/train", N_TRAIN)
process_split(X_val,   y_val,   "/kaggle/working/graphs/val",   N_VAL)
process_split(X_test,  y_test,  "/kaggle/working/graphs/test",  N_TEST)

# quick sanity: how many files did we write?
print("train files:", len(os.listdir("/kaggle/working/graphs/train")))
print("val files:",   len(os.listdir("/kaggle/working/graphs/val")))
print("test files:",  len(os.listdir("/kaggle/working/graphs/test")))


100%|██████████| 125/125 [00:15<00:00,  8.01it/s]

train files: 10000
val files: 2000
test files: 2000


In [9]:
import torch, os, random

sample_path = os.path.join("/kaggle/working/graphs/train", random.choice(os.listdir("/kaggle/working/graphs/train")))
g = torch.load(sample_path)
print(sample_path)
print("x:", g["x"].shape, g["x"].dtype)       # [num_nodes, 768] float16
print("edge_index:", g["edge_index"].shape)   # [2, num_edges]
print("y:", g["y"].item())


/kaggle/working/graphs/train/g_2773.pt
x: torch.Size([54, 768]) torch.float16
edge_index: torch.Size([2, 210])
y: 0


In [10]:
import os, glob, math, random, torch
import torch.nn as nn
import torch.nn.functional as F

# ---- dataset: loads your saved .pt graphs (x, edge_index, y) ----
class GraphDir(torch.utils.data.Dataset):
    def __init__(self, root, limit=None, shuffle=True):
        self.paths = sorted(glob.glob(os.path.join(root, "g_*.pt")))
        if shuffle:
            random.shuffle(self.paths)
        if limit is not None:
            self.paths = self.paths[:limit]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        g = torch.load(self.paths[idx])
        return g["x"].float(), g["edge_index"].long(), g["y"].long()

train_ds = GraphDir("/kaggle/working/graphs/train")   # optionally pass limit=2000 for speed
val_ds   = GraphDir("/kaggle/working/graphs/val")
test_ds  = GraphDir("/kaggle/working/graphs/test")

# simple batcher: process graphs one-by-one to keep it robust/lightweight
def iterate(dl):
    for item in dl:
        yield item

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=1, shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"


In [13]:
# --- GCN (pure torch) ---
import torch
import torch.nn as nn
import torch.nn.functional as F

def add_self_loops(edge_index, num_nodes):
    if edge_index.numel() == 0:
        return torch.stack([torch.arange(num_nodes), torch.arange(num_nodes)], dim=0)
    self_loops = torch.arange(num_nodes, device=edge_index.device)
    self_loops = torch.stack([self_loops, self_loops], dim=0)
    return torch.cat([edge_index, self_loops], dim=1)

class GCNConv(nn.Module):
    def __init__(self, in_dim, out_dim, bias=True):
        super().__init__()
        self.lin = nn.Linear(in_dim, out_dim, bias=bias)

    def forward(self, x, edge_index):
        # edge_index: [2, E] with src->dst
        N = x.size(0)
        edge_index = add_self_loops(edge_index, N)
        src, dst = edge_index

        # degree (with self-loops)
        deg = torch.bincount(dst, minlength=N).float().to(x.device)

        # normalized message passing: D^-1/2 A D^-1/2 X
        norm = 1.0 / torch.sqrt(torch.clamp(deg, min=1.0))
        x = self.lin(x)

        # aggregate
        out = torch.zeros_like(x)
        msg = x[src] * norm[src].unsqueeze(1)
        out.index_add_(0, dst, msg)
        out = out * norm.unsqueeze(1)
        return out

class GCNClassifier(nn.Module):
    def __init__(self, in_dim=768, hid=256, out_dim=2, dropout=0.2):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hid)
        self.conv2 = GCNConv(hid, hid)
        self.dropout = nn.Dropout(dropout)
        self.cls = nn.Linear(hid, out_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index); x = F.relu(x)
        x = self.conv2(x, edge_index); x = F.relu(x)
        g = x.mean(dim=0, keepdim=True)        # global mean pool
        g = self.dropout(g)
        return self.cls(g).squeeze(0)

# --- train/eval (same as before) ---
def run_epoch(model, loader, optim=None):
    is_train = optim is not None
    model.train(is_train)
    total_loss, correct, total = 0.0, 0, 0
    for x, edge_index, y in loader:
        x, edge_index, y = x[0].to(device), edge_index[0].to(device), y[0].to(device)
        if is_train: optim.zero_grad()
        logits = model(x, edge_index)
        loss = F.cross_entropy(logits.unsqueeze(0), y)
        if is_train:
            loss.backward()
            optim.step()
        total_loss += loss.item()
        correct += int(logits.argmax().item() == int(y.item()))
        total += 1
    return total_loss/max(total,1), correct/max(total,1)

model = GCNClassifier(in_dim=768, hid=256, out_dim=2, dropout=0.2).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

EPOCHS = 3
best_val = 0.0
for ep in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(model, train_loader, opt)
    va_loss, va_acc = run_epoch(model, val_loader, None)
    if va_acc > best_val:
        best_val = va_acc
        torch.save(model.state_dict(), "/kaggle/working/best_gcn.pt")
    print(f"epoch {ep}: train loss {tr_loss:.4f}, acc {tr_acc:.3f} | val acc {va_acc:.3f}")

# test
model.load_state_dict(torch.load("/kaggle/working/best_gcn.pt", map_location=device))
te_loss, te_acc = run_epoch(model, test_loader, None)
print(f"test acc {te_acc:.3f}")


epoch 1: train loss 0.2686, acc 0.943 | val acc 0.940
epoch 2: train loss 0.2789, acc 0.944 | val acc 0.940
epoch 3: train loss 0.2733, acc 0.944 | val acc 0.940
test acc 0.941
